#Instalação

In [ ]:
#!pip install yfinance
#!pip install crewai
!pip install crewai[tools]
#!pip install langchain
#!pip install langchain-openai
#!pip install duckduckgo-search
#!pip install streamlit
#!pip install langchain_community==0.0.29 langchain_groq

In [8]:
#Criando as Libs
#import json
import os
#import getpass
from datetime import datetime
from IPython.display import Markdown

import yfinance as yf

import openai
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchResults
import streamlit as st 

In [3]:
#Criando Yahoo Finance Tool
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-30", end="2024-08-30")
    #!stock = yf.download("AAPL", start="2023-08-31", end="2024-08-31")
    return stock

yahoo_finance_tool = Tool(
    name = 'Yahoo Finance Tool',
    description = """Fetches stocks prices for {ticket} from the Last year about a specific company from Yahoo Finance API""",
    func = lambda ticket: fetch_stock_price(ticket)
)

In [28]:
#Importando OPENAI LLM - GPT
#!from langchain_google_genai import ChatGoogleGenerativeAI
#!llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", verbose = True, max_output_tokens = 20, temperature = 0,timeout= None,google_api_key="API_KEY")
llm = ChatGroq(temperature=0,
    model_name="llama3-70b-8192",
    api_key='<API_KEY>')

In [29]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)
#!from langchain.tools import DuckDuckGoSearchRun
#!search_tool = DuckDuckGoSearchRun()

In [30]:
stockPriceAnalyst = Agent(
    role = 'Senior Stock Price Analyst',
    goal = 'Find the {ticket} stock price and analyses trends',
    backstory = """You're highly experienced in analyzing the price of an specific stock and make predictions about it's future price""",
    verbose= True,
    llm = llm,
    max_iter = 60,
    memory = True,
    tools=[yahoo_finance_tool],
    allow_delegation = False
)

In [31]:
getStockPrice = Task(
    description= """Analyze the stock {ticket} price history and create an trend analyzes of up, down or sideways""",
    expected_output = """ Specify the current stock price - up, down or sideways.
    eg. stock= 'APPL, price UP'
    """,
    agent= stockPriceAnalyst
)

In [32]:
newsAnalyst = Agent(
    role = 'Stock News Analyst',
    goal = 'Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with the news context. For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed',
    backstory = """You're highly experienced in analyzing the market trends and news and have tracked assets for more than 30 years.
    
    You're also master level analysts in the tradictional markets and have deep understanding of human psychology
    
    You understand news, theirs tittles and information, but you look at those with a health dose of skepticism.
    You consider also the source of the news articles.
    """,
    verbose= True,
    llm = llm,
    max_iter = 60,
    memory = True,
    tools=[search_tool],
    allow_delegation = False
)

In [33]:
get_news = Task(
    description= f"""Take the stock and always include BTC to it (if not request)
    Use the search tool to search each one individually
    
    The current date is {datetime.now()}.
    
    Compose the results into a helpful report""",
    expected_output = """A summary of the overall market and one sentence summary for each requested asset.
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent= newsAnalyst
)

In [34]:
stockAnalystWriter = Agent(
    role = 'Senior Stock Analysts Writer',
    goal = 'Analyze the trends price and news, writing an insightfull compelling and informative 3 paragraph long newsletter based on the stock report in price trend.',
    backstory = """You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories and narratives that resonates with wider audiences.
    
    You understand macro factors and combine multiple theories
    eg. cycle theory and fundamental analyses. You're able to hold multiple opinions when analyzing anything.""",
    verbose = True,
    max_iter = 60,
    memory = True,
    llm = llm,
    #!tools=[search_tool],
    allow_delegation = True
)

In [35]:
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary""",
    expected_output = """An eloquent 3 paragraph newsletter formated as markdown in an easy readable manner. It should contain:
    
    -3 bullet executive summary
    - Introduction - set the overall picture and spike up the interest
    - main part provides the most of the analysis including the news summary and fear/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways.
    """,
    agent = stockAnalystWriter,
    context = [getStockPrice, get_news]
)

In [36]:
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst],
    manager_agent=stockAnalystWriter,
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose = 1,
    process = Process.hierarchical,
    #!full_output=True,
    share_crew=False
    #!,manager_llm=llm
)

2024-09-14 13:53:47,269 - 4296 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
results = crew.kickoff(inputs={"ticket":'AAPL'})

In [ ]:
list(results.keys())

In [ ]:
results['final_ouput']

In [ ]:
len(results['tasks_output'])

In [ ]:
Markdown(results['final_output'])

In [ ]:
with open('result.txt', 'a') as fp:
    fp.write(result)

In [ ]:
#Declarando variavel sobre a função
response = yahoo_finance_tool.run('AAPL')
print(response)